**Unit 2 - Part 1a: LangChain Setup & Models**

In [6]:
%pip install python-dotenv --upgrade --quiet langchain langchain-google-genai

In [7]:
from dotenv import load_dotenv

In [8]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key:")

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Model A: The "Accountant" (Precision)
llm_focused = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.0)

#Modek B: The
llm_creative = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=1.0)

In [10]:
prompt = "Explain what an 'Idea' is in one sentence and contrast it with a 'Thought'."


print("--- FOCUSED (Temp=0) ---")
print(f"Run 1: {llm_focused.invoke(prompt).content}")
print(f"Run 2: {llm_focused.invoke(prompt).content}")

--- FOCUSED (Temp=0) ---
Run 1: An **idea** is a distinct mental concept, often novel, that serves as a solution, plan, or inspiration; in contrast, a **thought** is the broader, ongoing mental activity of processing information, reflecting, or considering various concepts.
Run 2: An **idea** is a distinct mental concept, often novel, that serves as a solution, plan, or inspiration; in contrast, a **thought** is the broader, ongoing mental activity of processing information, reflecting, or considering various concepts.


In [11]:
print("--- CREATIVE (Temp=1) ---")
print(f"Run 1: {llm_creative.invoke(prompt).content}")
print(f"Run 2: {llm_creative.invoke(prompt).content}")

--- CREATIVE (Temp=1) ---
Run 1: An **Idea** is a novel mental concept, plan, or insight that spontaneously originates in the mind, whereas a **Thought** is the broader, continuous process of mental activity, involving examination, reflection, and the processing of information or the exploration of ideas.
Run 2: An **idea** is a distinct, often novel, mental concept, plan, or image that can be developed or acted upon, while a **thought** is the broader, continuous process of mental activity itself, encompassing fleeting impressions, reflections, and the elaboration of ideas.


**Unit 2 - Part 1b: Prompts & Parsers**

In [12]:
#Setup from Part 1a (Hidden for brevity)
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API key:")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [13]:
#System message = controls tone + behavior
from langchain_core.messages import SystemMessage, HumanMessage

#Scenario: Make the AI rude.

messages = [
    SystemMessage(content="You are a Socratic teacher who never directly answers but guides the student with questions."),
    HumanMessage(content="What is the capital of France?")
]


response = llm.invoke(messages)
print(response.content)

What French cities come to mind when you think about the country?


In [14]:
from langchain_core.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate {input_language} to {output_language}."),
    ("human", "{text}")
])

# We can check what inputs it expects
print(f"Required variables: {template.input_variables}")

Required variables: ['input_language', 'output_language', 'text']


In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

# Raw Message
raw_msg = llm.invoke("Hi")
print(f"Raw Type: {type(raw_msg)}")

# Parsed String
clean_text = parser.invoke(raw_msg)
print(f"Parsed Type: {type(clean_text)}")
print(f"Content: {clean_text}")

Raw Type: <class 'langchain_core.messages.ai.AIMessage'>
Parsed Type: <class 'langchain_core.messages.base.TextAccessor'>
Content: Hi there! How can I help you today?


In [16]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter your Google API Key:")

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
template = ChatPromptTemplate.from_template("Tell me what you think about {topic}")
parser = StrOutputParser()

**Unit 2 - Part 1c: LCEL (LangChain Expression Language)**

In [17]:
prompt_value = template.invoke({"topic": "Consciousnes in space"})

response_obj = llm.invoke(prompt_value)

final_text = parser.invoke(response_obj)

print(final_text)

Consciousness in space is a truly fascinating and multi-layered concept, blending psychology, philosophy, astrobiology, and even a touch of the metaphysical. Here's what I think about it:

1.  **Human Consciousness in Space: The Immediate Reality**
    *   **The Overview Effect:** This is perhaps the most well-documented and profound impact of space on human consciousness. Astronauts who see Earth from orbit often report a cognitive shift – a deep understanding of the planet's fragility, interconnectedness, and the arbitrary nature of human-made borders. It's an existential shift, fostering a sense of universal belonging and responsibility. Their consciousness expands to encompass a global, rather than merely local, perspective.
    *   **Isolation and Confinement:** On the flip side, long-duration space missions (like on the ISS or future Mars missions) pose immense psychological challenges. The consciousness of an astronaut must cope with extreme isolation, sensory deprivation (relat

In [18]:
chain = template | llm | parser

print(chain.invoke({"topic": "Cats"}))

As an AI, I don't *think* or have personal opinions or feelings like humans do. However, I can tell you what I've learned about cats based on the vast amount of information I've processed, summarizing common human perspectives and facts!

From a human perspective, cats are truly fascinating and beloved creatures for many reasons:

1.  **Unique Blend of Independence and Affection:** Cats are often described as independent, and they certainly have a self-sufficient air about them. They often seem to do things on their own terms. Yet, they can also be incredibly affectionate, seeking out cuddles, purring on laps, rubbing against legs, and offering comforting companionship. This balance is a big part of their appeal.

2.  **Grace and Agility:** Watching a cat move is often a masterclass in silent, fluid motion. Their agility, balance, and ability to jump, climb, and squeeze into tight spaces are remarkable. Their natural predatory instincts, even in the most pampered housecat, are evident 

**Assignment**

Create a chain that:

Takes a movie name.
Asks for its release year.
Calculates how many years ago that was (You can try just asking the LLM to do the math).
Try to do it in one line of LCEL.

In [19]:
template = ChatPromptTemplate.from_template(
    "Determine the release year of the movie '{movie}'. "
    "Assume the current year is {current_year}. "
    "First compute the difference between {current_year} and the release year.\n\n"
    "Respond EXACTLY like this:\n"
    "{movie} was released in YEAR.\n"
    "It was released X years ago."
)



In [20]:
chain = template | llm | parser

print(chain.invoke({
    "movie": "Interstellar",
    "current_year": 2026
}))


Interstellar was released in 2014.
It was released 12 years ago.
